In [1]:
# This code shows example function calls and walks through Sync Module Code. It loads in the current parameter files
# and the module code. Code here will walk through the "main" function and call the helper functions in "sync.jl"

# First section of code here will just load in parameters and orbits
#Note: will need to change file locations to load in data
include("/Users/loria/Documents/GitHub/darts-simtool/input_parameters_sync.jl") # clock/sync system parameters
include("/Users/loria/Documents/GitHub/darts-simtool/input_parameters_1.jl") # radar system parameters
include("/Users/loria/Documents/GitHub/darts-simtool/sync_helper_functions.jl") # file with the ancillary functions for sync module

#import Pkg # if not already installed
#Pkg.add("Interpolations")
#Pkg.add("LinearAlgebra")
#Pkg.add("FFTW")
#Pkg.add("SatelliteToolbox")
#Pkg.add("NCDatasets")
#Pkg.add("Plots")
#Pkg.add("Dates")

using Interpolations
using LinearAlgebra
using FFTW
using NCDatasets
using Plots
using Dates

# Introduce orbit inputs... need positions and timing. Use example code from Razi (also interpolate to kHz-level radar PRI)
include("/Users/loria/Documents/GitHub/darts-simtool/modules/orbits.jl")



# Create NetCDF data structure
ds = Dataset("/Users/loria/Desktop/DARTS/Julia Codes/orbitOutput_082020.nc");


# read in time and position data
read_samps = 1:1500;
time_vector = ds["time"][read_samps];
pos = ds["position"][:,:,read_samps];


# verify size of position input and get number of platforms.
szp = size(pos)
@assert szp[1]==3 "POS needs to be 3 x [Np x Nt] or 3 x Nt"
if ndims(pos) == 3
    nplat = szp[2]
    ntimes = szp[2]
elseif ndims(pos) == 2
    nplat = 1
    ntimes = szp[2]
end

# test plot of orbits
plot(time_vector, pos[:,1,:]', xaxis=("time (sec)"), ylabel=("ECEF position (km)"))

# nominally I think we'd interpolate orbits (either before in orbits module or here) to the PRI times
# for now, I'll just leave it at the original orbit sampling.

LoadError: could not open file /Users/loria/Documents/GitHub/darts-simtool/sync_helper_functions.jl

In [2]:
# Variables defined here are helpers in the sync module
# find total elapsed time over the course of the orbits
t_elapse = time_vector[end] - time_vector[1]
sdradar_args_fs = round(clk_args_N/t_elapse) # upper bound of frequency in the PSD generation. Limited (i.e. selected) by the sync_pri
clk_args_N = ceil(sdradar_args_fs*t_elapse)  # Number of sample points in the PSD. Needs to meet the length of min {1/sync_pri,t_elapse}
m = fc/clk_args_f_osc;          # frequency up-conversion factor (scale factor from LO to RF)

# sync module has if-else statement to consider a single platform. For this example we assume multiple platforms for the example

LoadError: UndefVarError: time_vector not defined

In [3]:
# create matrix of phase error values (allocate memory)
phase_err = Complex(nplat,ntimes) # Nplatforms x Ntimes

#for i = 1:nplat #for each platform, generate oscillator phase errors at each time point
i=2 # only calculate second platform for example

a_coeff_db = osc_coeffs[i,:] # grab the clock coefficients for each platform


LoadError: UndefVarError: ntimes not defined

In [4]:
# find CRLB based on SNR of intersatellite link. Returns both crlb and the SNR matrices
(sig_crlb, snr_dB) = getSensorCRLB(pos,sdradar_args_N,sdradar_args_fc,sdradar_args_fs,sdradar_args_fbw,master)

#plot()
plot(time_vector, sig_crlb[2,:].*1e9, xaxis=("time (sec)"), ylabel=("CRLB"))
#plot(time_vector, snr_dB[2,:], xaxis=("time (sec)"), ylabel=("SNR (dB)"))




LoadError: UndefVarError: getSensorCRLB not defined

In [5]:
# here is the first call of a helper function, "osc_psd_twosided." Generates PSD without the effects of synchronization
(Sphi,f_psd) = osc_psd_twosided(sdradar_args_fs,clk_args_N,a_coeff_db)

plot(f_psd,Sphi,xaxis=("frequency (Hz)"), ylabel=("PSD Height"))

size(Sphi)


LoadError: UndefVarError: osc_psd_twosided not defined

In [6]:
# calls function to model sync effects on PSD
# in the sync module we loop through this call to capture the sync for each platform. 
# Here, we're only showing a single call for one platform
Sphi_sync = sync_effects_on_PSD(Sphi,f_psd,sync_radar_offset,sig_crlb[i,:],sync_pri,sdradar_args_fs)

plot(f_psd,Sphi_sync,xaxis=("frequency (Hz)"), ylabel=("PSD Height after Sync"))

LoadError: UndefVarError: sig_crlb not defined

In [7]:
# generate time series of phase error
(r,t) = osc_timeseries_from_psd_twosided(Sphi_sync,sdradar_args_fs)

# interpolate phase error to the measurement times
itp = CubicSplineInterpolation(t, r) # create interpolant
phase = itp(time_vector)             # interpolates to the time vector given by orbits module
phase_err[i,:] = phase.*m            # scale by frequency up-conversion factor

plot(time_vector,phase_err[i,:],xaxis=("Time (s)"), ylabel=("Phase Error"))

# this phase error will be an input to generate_raw_data.jl 
# within generate_raw_data.jl, the phase error (after sync) is modeled as an extra term ...*exp(-im*phase_error)

LoadError: UndefVarError: osc_timeseries_from_psd_twosided not defined